<a href="https://colab.research.google.com/github/Pengwei-Yang/Robust-to-noisy-data/blob/main/Robust_to_label_noise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Load data

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import os
import random
import numpy as np
from PIL import Image
from collections import OrderedDict

In [ ]:
class A2Dataset(Dataset):
    r"""Self-defined child class of the pytorch Dataset object.
    """
    def __init__(self, data, labels, transform=None):
        self.data = np.asarray(data)
        self.labels = np.asarray(labels)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data = self.data[idx]
        label = self.labels[idx]
        #Img should be PIL Image
        #Transfrom numpy array to PIL
        if len(data.shape) == 3:
            data = Image.fromarray(np.uint8(data)).convert('RGB')
        else:
            data = Image.fromarray(np.uint8(data)).convert('L')

        if self.transform is not None:
              data = self.transform(data)
        return (data, label)

def train_valid_random_split(data, fracs, labels):
    r"""Splitting data to training data and validation data.
    Args:
        data: 3 dimension, E.g.,(18000, 28, 28)
        labels: 1 dimension, E.g.,(18000,)
    Returns:
        shuffled_data: 3 dimension
        shuffled_labels: 1 dimension
    """
    #Assert for debugging
    assert len(fracs) == 2
    assert sum(fracs) == 1
    assert all(frac > 0 for frac in fracs)
    nn = len(data)
    subset_lens = [int(frac*nn) for frac in fracs]
    idxs = list(range(nn))
    #Shuffle index
    random.shuffle(idxs)
    shuffled_data = []
    shuffled_labels=[]
    start_idx = 0
    #Shuffle data and labels/ or using np.random.permutation() + SubsetRandomSampler()
    for subset_len in subset_lens:
        end_idx = start_idx + subset_len
        cur_idxs = idxs[start_idx:end_idx]
        shuffled_data.append(data[cur_idxs,:,:].tolist())
        shuffled_labels.append(labels[cur_idxs].tolist())
        start_idx = end_idx
    return shuffled_data, shuffled_labels

def get_loader(batch_size = 128, num_workers = 1, proportion = [0.8,0.2], file_path=None, transform=None):
    r"""This function aims to read the data file and split the data into two subsets, i.e,
    train data, validation data, and return their DataLoader objects.
    Args:
        file_path: path of the dataset
        batch_size: one of the hyperparameters 128 or 64 or ...
        num_workers: amount of threads, default=0->main thread
        transfrom: Data preprocessing
    """
    #Split and transform data
    #[train_data, val_data] = train_valid_random_split(train_val_data, fracs = proportion)
    dataset = np.load(file_path)

    Xtr_val = dataset['Xtr']
    Str_val = dataset['Str']
    Xts = dataset['Xts']
    Yts = dataset['Yts']

    train_val_data = Xtr_val
    train_labels = Str_val
    test_data = Xts
    test_labels = Yts

    shuffled_data, shuffled_labels = train_valid_random_split(data=train_val_data, labels=train_labels, fracs = proportion)
    train_data = shuffled_data[0]
    val_data = shuffled_data[1]
    train_label = shuffled_labels[0]
    test_label = shuffled_labels[1]

    train_data = A2Dataset(data = train_data, labels=train_label, transform=transform)
    val_data = A2Dataset(data = val_data, labels=test_label, transform=transform)
    test_data = A2Dataset(data = test_data, labels=test_labels, transform=transform)

    #DataLoader-->Return batch of data, choose number of threads etc.
    train_loader = DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=False,
    )

    val_loader = DataLoader(
        val_data,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=False,
    )

    test_loader = DataLoader(
        test_data,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=False,
    )
    return train_loader, val_loader, test_loader

In [ ]:
#LeNet-5 Backbone
#Defining the convolutional neural network
class LeNet5(nn.Module):
    def __init__(self, num_class, transition_matrix=None):
        super(LeNet5, self).__init__()

        self.transition_matrix = transition_matrix
        # if self.transition_matrix is None:
        #     self.transition_matrix = torch.eye(num_class)
        # if self.transition_matrix is not None:
        #     assert(num_class == transition_matrix.shape[0] and num_class == transition_matrix.shape[1] and num_class == transition_matrix.shape[2])

        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.classifier = nn.Sequential(
            nn.Linear(400, 120),
            nn.ReLU(),  #nn.ReLU(inplace=True) can sometimes save memory, but destroy input data
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, num_class)
        )

    def backward_learning(self, x):
        if self.transition_matrix != None:
            if x.is_cuda:
                y_clean = torch.mm(self.transition_matrix.cuda().inverse(), x.T).T
            else:
                y_clean = torch.mm(self.transition_matrix.inverse(), x.T).T
            return y_clean
        else:
            return x

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.shape[0], -1) #flatten
        out = self.classifier(out)
        out = self.backward_learning(out) #Noisy posterior-->Clean posterior
        return out






In [ ]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=3, transition_matrix=None):
        super(AlexNet, self).__init__()

        self.transition_matrix = transition_matrix

        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 96, kernel_size=11, stride=4, padding=0),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer5 = nn.Sequential(
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(9216, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, num_classes))

    def backward_learning(self, x):
        if self.transition_matrix != None:
            if x.is_cuda:
                y_clean = torch.mm(self.transition_matrix.cuda().inverse(), x.T).T
            else:
                y_clean = torch.mm(self.transition_matrix.inverse(), x.T).T
            return y_clean
        else:
            return x

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.backward_learning(out)
        return out

In [ ]:
# When all random seeds are fixed, the python runtime environment becomes deterministic.
def seed_torch(seed=1000):
    r"""Fix all random seeds for repeating the expriement result."""
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # If multi-GPUs are used.
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


# Embedding labels to one-hot form.
def one_hot_embedding(labels, num_classes):
    r"""Embedding labels to one-hot form.

    Args:
      labels: (LongTensor) class labels, sized [N,].
      num_classes: (int) number of classes.

    Returns:
      (tensor) encoded labels, sized [N, #classes].
    """
    y = torch.eye(num_classes)
    return y[labels]


# Calcuate the accuracy according to the prediction and the true label.
def accuracy(output, target, topk=(1,)):
    r"""Computes the precision@k for the specified values of k."""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


# A helper function which is used to record the experiment results.
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, num):
        self.val = val
        self.sum += val * num
        self.count += num
        self.avg = self.sum / self.count


# # Load a NN model.
# def load_model(m_config):
#     model = globals()[m_config['model_name']](
#         input_dim=m_config["input_dim"],
#         hidden_dim=m_config["hidden_dim"],
#         output_dim=m_config["output_dim"]
#     )
#     return model

#Training and validation functions

In [ ]:
def train(epoch, model, optimizer, criterion, train_loader):
    """Training a pytorch nn_model."""
    top1_acc_meter = AverageMeter()
    loss_meter = AverageMeter()

    # swith model to to train mode
    model.train()
    for step, (data, targets) in enumerate(train_loader):
        # prepare min_batch
        data = data.to(device)
        targets = targets.to(device)

        # predict
        preds = model(data)

        # forward
        loss = criterion(preds, targets)

        # set all gradients to zero
        optimizer.zero_grad()

        # backward
        loss.backward()

        # update all gradients
        optimizer.step()

        # calculate accuracy
        [top1_acc] = accuracy(preds.data, targets.data, topk=(1,))
        # record accuary and cross entropy losss
        min_batch_size = data.size(0)
        top1_acc_meter.update(top1_acc.item(), min_batch_size)
        loss_meter.update(loss.item(), min_batch_size)

    print("Train ","accuracy->%.2f" %top1_acc_meter.avg, "loss->%.2f" %loss_meter.avg)


def validate_and_test(epoch, model, criterion, val_loader, is_test=False):
    """Validation or testing of a nn_model."""
    top1_acc_meter = AverageMeter()
    loss_meter = AverageMeter()

    # swith model to to eval mode
    # if is_test==False:

    model.eval()
    for step, (data, targets) in enumerate(val_loader): #in this val & test function, we dont write backward here
        # prepare min_batch
        data = data.to(device)
        targets = targets.to(device)

        # predict
        with torch.no_grad():
            preds = model(data)

        # forward
        loss = criterion(preds, targets)

        # calculate accuracy
        [top1_acc] = accuracy(preds.data, targets.data, topk=(1,))

        # record accuary and cross entropy losss
        min_batch_size = data.size(0)
        top1_acc_meter.update(top1_acc.item(), min_batch_size)
        loss_meter.update(loss.item(), min_batch_size)

    top1_acc_avg = top1_acc_meter.avg
    loss_avg = loss_meter.avg
    if(is_test == False):
        print("Validate ", "val_accuracy-->%.2f" %top1_acc_avg, "val_loss-->%.2f" %loss_avg)
    else:
        print("Test Accuracy--->%.2f" %top1_acc_avg, "Test Loss--->%.2f" %loss_avg)

    return top1_acc_avg

In [ ]:
#Run training and validation
# Some global variabiles
device = None
train_loader = None
model = None
# model_config = OrderedDict([
#     ('model_name','Predefined_FCNet'),
#     ('input_dim',10),
#     ('hidden_dim',2),
#     ('output_dim',2)
# ])

optim_config = OrderedDict([
    #('epochs', 15),
    ('batch_size', 128),
    #('base_lr', 0.1),
    ('weight_decay', 1e-5),
    ('momentum', 0.9),
    # ('lr_decay', 0.1)
])

run_config = OrderedDict([
    ('seed', 1),
    ('outdir', 'trained_model'),
    ('num_workers', 1),
])

# data_config = OrderedDict([
#     ('flip_rates', [0.15,0.25]),
#     ('data_path', "./input.data"),
#     ('train_val_test_split', [0.7,0.1,0.2])
# ])

net_parameters = OrderedDict([
    # ('num_epochs', 10),
    ('batch_size', 128),
    ('lr', 0.002),
    ('num_classes', 3)
])

config = OrderedDict([
    # ('model_config', model_config),
    ('optim_config', optim_config),
    ('run_config', run_config),
    # ('data_config', data_config),
    ('net_parameters', net_parameters)
])

# #Initialize model parameters
    # takes in a module and applies the specified weight initialization
# def weights_init_uniform_rule(m):
#     classname = m.__class__.__name__
#     # for every Linear layer in a model..
#     if classname.find('Linear') != -1:
#         # get the number of the inputs
#         n = m.in_features
#         y = 1.0/np.sqrt(n)
#         m.weight.data.uniform_(-y, y)
#         m.bias.data.fill_(0)

# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def run_train_val(train_loader, val_loader, test_loader, model_name, transition_matrix=None, lr=None, epoch_num=None):
    global device
    global config
    # global train_loader
    global model
    best_top1_acc = 0
    # check gpu
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # configerations
    run_config = config['run_config']
    optim_config = config['optim_config']
    # data_config = config['data_config']
    net_parameters = config['net_parameters']

    # set random seed->make the trained net of every turn different
    seed_torch(run_config['seed'])

    # create output directory
    outdir = run_config['outdir']
    if not os.path.exists(outdir):
        os.makedirs(outdir)

    # load data
    # data = pd.read_csv(data_config["data_path"], header = None).values.tolist()
    # data = binary_CCN_generator(data = data, flip_rates = data_config["flip_rates"])

    # train_loader, val_loader, test_loader = get_loader(
    #     batch_size = optim_config['batch_size'],
    #     num_workers = run_config['num_workers'],
    #     train_val_test_split = data_config['train_val_test_split'],
    #     data = data
    # )
    # train_loader = train_loader
    # val_loader = val_loader
    # test_loader = test_loader

    #Model
    if model_name == 'LeNet':
        model = LeNet5(net_parameters['num_classes'], transition_matrix=transition_matrix)
    elif model_name == 'AlexNet':
        model = AlexNet(net_parameters['num_classes'], transition_matrix=transition_matrix)
    else:
        print('Input Model Name Error!!!')
        return 0
    if torch.cuda.device_count() > 1:
        print(torch.cuda.device_count(), "GPUs are used!")
        model = nn.DataParallel(model)
    model.to(device)



    # criterion and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(
        params = model.parameters(),
        lr=lr,
        momentum = optim_config['momentum'],
        weight_decay = optim_config['weight_decay']
    )
    print("【New turn...】")



    for epoch in range(1, epoch_num + 1):
        print("Epoch", epoch, "/", epoch_num)
        #train
        train(epoch, model, optimizer, criterion, train_loader)

        #validation
        top1_acc_avg = validate_and_test(epoch, model, criterion, val_loader)


        # save the best model so far
        if (top1_acc_avg > best_top1_acc):
            state = OrderedDict([
                ('config', config),
                ('state_dict', model.state_dict()),
                ('optimizer', optimizer.state_dict()),
                ('epoch', epoch),
                ('top1-accuracy', top1_acc_avg),
            ])
            best_model_path = os.path.join(outdir, 'model_best.pth')
            torch.save(state, best_model_path)
            best_top1_acc = top1_acc_avg
    #test->for one epoch
    test_acc_avg = validate_and_test(epoch, model, criterion, test_loader, is_test=True) #test function without backpropogation



    # return test_acc_avg
    # model = LeNet5(net_parameters['num_classes'], transition_matrix=transition_matrix)
    # model.load_state_dict(torch.load(best_model_path))
    # model.eval()
    # #model = torch.load(best_model_path)
    # #test
    # print(type(model))




##MINIST-LeNet+ALexNet

In [ ]:
#Choose transition matrix and input size based on input name --> MINIST05/MINIST06 and LeNet/AlexNet
def MINIST(transform=None, batch_size=128, name=None):
    file_path = None
    print('Code Name: ', name)
    if name == 'MINIST05_LeNet':
        print('Setting: MINIST05_LeNet activated!')
    #Transition matrix of MINIST05
        transition_matrix = torch.Tensor(
            [[0.5, 0.2, 0.3],
            [0.3, 0.5, 0.2],
            [0.2, 0.3, 0.5]]
            )

        file_path = "/content/FashionMNIST0.5.npz" #Input your own path of FashionMNIST0.5 here

        #transform-Data preprocessing
        if transform is None:
            transform = transforms.Compose([
                transforms.Resize((32,32)),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.ToTensor(),
                transforms.Normalize((0.5),(0.5)) #Only one channel (Grey image)[-1,+1]
            ])

    if name == 'MINIST05_AlexNet':
        print('Setting: MINIST05_AlexNet activated!')
          #Transition matrix of MINIST05
        transition_matrix = torch.Tensor(
            [[0.5, 0.2, 0.3],
            [0.3, 0.5, 0.2],
            [0.2, 0.3, 0.5]]
            )

        file_path = "/content/FashionMNIST0.5.npz" #Input your own path of FashionMNIST0.5 here

        #transform-Data preprocessing
        if transform is None:
            transform = transforms.Compose([
                transforms.Resize((227,227)),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.ToTensor(),
                transforms.Normalize((0.5),(0.5)) #Only one channel (Grey image)[-1,+1]
            ])

    if name == 'MINIST06_LeNet':
        print('Setting: MINIST06_LeNet activated!')
        #Transition matrix of MINIST06
        transition_matrix = torch.Tensor(
            [[0.4, 0.3, 0.3],
            [0.3, 0.4, 0.3],
            [0.3, 0.3, 0.4]]
            )

        file_path = "/content/FashionMNIST0.6.npz" #Input your own path of FashionMNIST0.5 here

        #transform-Data preprocessing
        if transform is None:
            transform = transforms.Compose([
                transforms.Resize((32,32)),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.ToTensor(),
                transforms.Normalize((0.5),(0.5)) #Only one channel (Grey image)[-1,+1]
            ])

    if name == 'MINIST06_AlexNet':
        print('Setting: MINIST06_AlexNet activated!')
        #Transition matrix of MINIST06
        transition_matrix = torch.Tensor(
            [[0.4, 0.3, 0.3],
            [0.3, 0.4, 0.3],
            [0.3, 0.3, 0.4]]
            )

        file_path = "/content/FashionMNIST0.6.npz" #Input your own path of FashionMNIST0.5 here

        #transform-Data preprocessing
        if transform is None:
            transform = transforms.Compose([
                transforms.Resize((227,227)),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.ToTensor(),
                transforms.Normalize((0.5),(0.5)) #Only one channel (Grey image)[-1,+1]
            ])

    train_loader, val_loader, test_loader = get_loader(file_path=file_path, transform=transform,batch_size=batch_size)

    return transition_matrix, train_loader, val_loader, test_loader

#Training and test

##Classfier- LeNet

####DATASET: MINIST05

####MODEL NAME: LeNet

####NAME CODE: MINIST05_LeNet

####STATUS: Without Backward Learning

In [ ]:
transition_matrix_M5_Le_N, train_M5_Le_N, val_M5_Le_N, test_M5_Le_N = MINIST(batch_size=512, name='MINIST05_LeNet')
run_train_val(train_loader=train_M5_Le_N, val_loader=val_M5_Le_N, test_loader=test_M5_Le_N, model_name='LeNet', lr=0.08, epoch_num=10)

Code Name:  MINIST05_LeNet
Setting: MINIST05_LeNet activated!
【New turn...】
Epoch 1 / 10
Train  accuracy->43.49 loss->1.07
Validate  val_accuracy-->46.25 val_loss-->1.06
Epoch 2 / 10
Train  accuracy->47.32 loss->1.05
Validate  val_accuracy-->48.42 val_loss-->1.05
Epoch 3 / 10
Train  accuracy->47.64 loss->1.05
Validate  val_accuracy-->48.39 val_loss-->1.05
Epoch 4 / 10
Train  accuracy->48.12 loss->1.04
Validate  val_accuracy-->48.61 val_loss-->1.05
Epoch 5 / 10
Train  accuracy->48.43 loss->1.04
Validate  val_accuracy-->48.58 val_loss-->1.05
Epoch 6 / 10
Train  accuracy->48.34 loss->1.04
Validate  val_accuracy-->48.42 val_loss-->1.05
Epoch 7 / 10
Train  accuracy->48.40 loss->1.04
Validate  val_accuracy-->48.36 val_loss-->1.05
Epoch 8 / 10
Train  accuracy->48.73 loss->1.04
Validate  val_accuracy-->48.42 val_loss-->1.05
Epoch 9 / 10
Train  accuracy->48.69 loss->1.04
Validate  val_accuracy-->48.17 val_loss-->1.05
Epoch 10 / 10
Train  accuracy->48.90 loss->1.04
Validate  val_accuracy-->48.53

####DATASET: MINIST05

####MODEL NAME: LeNet

####NAME CODE: MINIST05_LeNet

####STATUS: With Backward Learning

In [ ]:
transition_matrix_M5_Le_Y, train_M5_Le_Y, val_M5_Le_Y, test_M5_Le_Y = MINIST(batch_size=512, name='MINIST05_LeNet')
run_train_val(train_loader=train_M5_Le_Y, val_loader=val_M5_Le_Y, test_loader=test_M5_Le_Y, model_name='LeNet', transition_matrix=transition_matrix_M5_Le_Y, lr=0.08, epoch_num=9)

Code Name:  MINIST05_LeNet
Setting: MINIST05_LeNet activated!
【New turn...】
Epoch 1 / 9
Train  accuracy->41.12 loss->1.10
Validate  val_accuracy-->42.67 val_loss-->1.08
Epoch 2 / 9
Train  accuracy->46.24 loss->1.06
Validate  val_accuracy-->47.58 val_loss-->1.06
Epoch 3 / 9
Train  accuracy->46.84 loss->1.05
Validate  val_accuracy-->47.83 val_loss-->1.06
Epoch 4 / 9
Train  accuracy->47.76 loss->1.05
Validate  val_accuracy-->47.86 val_loss-->1.06
Epoch 5 / 9
Train  accuracy->48.03 loss->1.05
Validate  val_accuracy-->48.03 val_loss-->1.06
Epoch 6 / 9
Train  accuracy->48.24 loss->1.05
Validate  val_accuracy-->48.61 val_loss-->1.06
Epoch 7 / 9
Train  accuracy->48.35 loss->1.05
Validate  val_accuracy-->48.61 val_loss-->1.06
Epoch 8 / 9
Train  accuracy->48.44 loss->1.04
Validate  val_accuracy-->48.36 val_loss-->1.05
Epoch 9 / 9
Train  accuracy->48.76 loss->1.04
Validate  val_accuracy-->48.50 val_loss-->1.06
Test Accuracy--->91.30 Test Loss--->0.76


####DATASET: MINIST06

####MODEL NAME: LeNet

####NAME CODE: MINIST06_LeNet

####STATUS: Without Backward Learning

In [ ]:
transition_matrix_M6_Le_N, train_M6_Le_N, val_M6_Le_N, test_M6_Le_N = MINIST(batch_size=128, name='MINIST06_LeNet')
run_train_val(train_loader=train_M6_Le_N, val_loader=val_M6_Le_N, test_loader=test_M6_Le_N, model_name='LeNet', lr=0.0025, epoch_num=6)

Code Name:  MINIST06_LeNet
Setting: MINIST06_LeNet activated!
【New turn...】
Epoch 1 / 6
Train  accuracy->36.64 loss->1.10
Validate  val_accuracy-->37.36 val_loss-->1.09
Epoch 2 / 6
Train  accuracy->38.55 loss->1.09
Validate  val_accuracy-->37.11 val_loss-->1.09
Epoch 3 / 6
Train  accuracy->39.24 loss->1.09
Validate  val_accuracy-->36.97 val_loss-->1.09
Epoch 4 / 6
Train  accuracy->39.26 loss->1.09
Validate  val_accuracy-->36.92 val_loss-->1.09
Epoch 5 / 6
Train  accuracy->39.24 loss->1.09
Validate  val_accuracy-->37.39 val_loss-->1.09
Epoch 6 / 6
Train  accuracy->39.41 loss->1.09
Validate  val_accuracy-->37.75 val_loss-->1.09
Test Accuracy--->87.57 Test Loss--->0.97


####DATASET: MINIST06

####MODEL NAME: LeNet

####NAME CODE: MINIST06_LeNet

####STATUS: With Backward Learning

In [ ]:
transition_matrix_M6_Le_Y, train_M6_Le_Y, val_M6_Le_Y, test_M6_Le_Y = MINIST(batch_size=128, name='MINIST06_LeNet')
run_train_val(train_loader=train_M6_Le_Y, val_loader=val_M6_Le_Y, test_loader=test_M6_Le_Y, model_name='LeNet', transition_matrix=transition_matrix_M6_Le_Y, lr=0.0025, epoch_num=6)

Code Name:  MINIST06_LeNet
Setting: MINIST06_LeNet activated!
【New turn...】
Epoch 1 / 6
Train  accuracy->35.47 loss->1.12
Validate  val_accuracy-->36.56 val_loss-->1.10
Epoch 2 / 6
Train  accuracy->36.82 loss->1.10
Validate  val_accuracy-->37.11 val_loss-->1.10
Epoch 3 / 6
Train  accuracy->37.32 loss->1.10
Validate  val_accuracy-->37.39 val_loss-->1.10
Epoch 4 / 6
Train  accuracy->37.81 loss->1.09
Validate  val_accuracy-->37.22 val_loss-->1.10
Epoch 5 / 6
Train  accuracy->38.18 loss->1.09
Validate  val_accuracy-->37.03 val_loss-->1.10
Epoch 6 / 6
Train  accuracy->38.31 loss->1.09
Validate  val_accuracy-->37.81 val_loss-->1.10
Test Accuracy--->82.50 Test Loss--->0.97


##Classifier- AlexNet

####DATASET: MINIST05

####MODEL NAME: AlexNet

####NAME CODE: MINIST05_AlexNet

####STATUS: Without Backward Learning

In [ ]:
transition_matrix_M5_Alex_N, train_M5_Alex_N, val_M5_Alex_N, test_M5_Alex_N = MINIST(batch_size=128, name='MINIST05_AlexNet')
run_train_val(train_loader=train_M5_Alex_N, val_loader=val_M5_Alex_N, test_loader=test_M5_Alex_N, model_name='AlexNet', lr=0.0002, epoch_num=10)

Code Name:  MINIST05_AlexNet
Setting: MINIST05_AlexNet activated!
【New turn...】
Epoch 1 / 10
Train  accuracy->37.92 loss->1.11
Validate  val_accuracy-->45.36 val_loss-->1.06
Epoch 2 / 10
Train  accuracy->42.01 loss->1.08
Validate  val_accuracy-->46.44 val_loss-->1.06
Epoch 3 / 10
Train  accuracy->42.88 loss->1.08
Validate  val_accuracy-->47.00 val_loss-->1.05
Epoch 4 / 10
Train  accuracy->43.94 loss->1.07
Validate  val_accuracy-->47.11 val_loss-->1.05
Epoch 5 / 10
Train  accuracy->44.63 loss->1.07
Validate  val_accuracy-->47.42 val_loss-->1.05
Epoch 6 / 10
Train  accuracy->44.64 loss->1.07
Validate  val_accuracy-->47.97 val_loss-->1.05
Epoch 7 / 10
Train  accuracy->45.17 loss->1.06
Validate  val_accuracy-->47.94 val_loss-->1.05
Epoch 8 / 10
Train  accuracy->45.42 loss->1.06
Validate  val_accuracy-->48.08 val_loss-->1.05
Epoch 9 / 10
Train  accuracy->45.31 loss->1.06
Validate  val_accuracy-->48.06 val_loss-->1.05
Epoch 10 / 10
Train  accuracy->45.37 loss->1.06
Validate  val_accuracy-->4

####DATASET: MINIST05

####MODEL NAME: AlexNet

####NAME CODE: MINIST05_AlexNet

####STATUS: With Backward Learning

In [ ]:
transition_matrix_M5_Alex_Y, train_M5_Alex_Y, val_M5_Alex_Y, test_M5_Alex_Y = MINIST(batch_size=128, name='MINIST05_AlexNet')
run_train_val(train_loader=train_M5_Alex_Y, val_loader=val_M5_Alex_Y, test_loader=test_M5_Alex_Y, model_name='AlexNet', transition_matrix=transition_matrix_M5_Alex_Y, lr=0.0002, epoch_num=10)

Code Name:  MINIST05_AlexNet
Setting: MINIST05_AlexNet activated!
【New turn...】
Epoch 1 / 10
Train  accuracy->37.49 loss->1.31
Validate  val_accuracy-->46.56 val_loss-->1.06
Epoch 2 / 10
Train  accuracy->39.11 loss->1.16
Validate  val_accuracy-->47.03 val_loss-->1.06
Epoch 3 / 10
Train  accuracy->40.49 loss->1.12
Validate  val_accuracy-->47.47 val_loss-->1.05
Epoch 4 / 10
Train  accuracy->41.56 loss->1.10
Validate  val_accuracy-->47.33 val_loss-->1.05
Epoch 5 / 10
Train  accuracy->42.73 loss->1.09
Validate  val_accuracy-->47.86 val_loss-->1.05
Epoch 6 / 10
Train  accuracy->42.35 loss->1.09
Validate  val_accuracy-->47.58 val_loss-->1.05
Epoch 7 / 10
Train  accuracy->43.92 loss->1.08
Validate  val_accuracy-->47.58 val_loss-->1.05
Epoch 8 / 10
Train  accuracy->43.72 loss->1.07
Validate  val_accuracy-->48.11 val_loss-->1.05
Epoch 9 / 10
Train  accuracy->44.03 loss->1.07
Validate  val_accuracy-->48.28 val_loss-->1.05
Epoch 10 / 10
Train  accuracy->44.62 loss->1.07
Validate  val_accuracy-->4

####DATASET: MINIST06

####MODEL NAME: AlexNet

####NAME CODE: MINIST06_AlexNet

####STATUS: Without Backward Learning

In [ ]:
transition_matrix_M6_Alex_N, train_M6_Alex_N, val_M6_Alex_N, test_M6_Alex_N = MINIST(batch_size=128, name='MINIST06_AlexNet')
run_train_val(train_loader=train_M6_Alex_N, val_loader=val_M6_Alex_N, test_loader=test_M6_Alex_N, model_name='AlexNet', lr=0.001, epoch_num=5)

Code Name:  MINIST06_AlexNet
Setting: MINIST06_AlexNet activated!
【New turn...】
Epoch 1 / 5
Train  accuracy->34.35 loss->1.13
Validate  val_accuracy-->37.25 val_loss-->1.10
Epoch 2 / 5
Train  accuracy->34.72 loss->1.11
Validate  val_accuracy-->37.69 val_loss-->1.09
Epoch 3 / 5
Train  accuracy->36.38 loss->1.11
Validate  val_accuracy-->37.58 val_loss-->1.09
Epoch 4 / 5
Train  accuracy->36.55 loss->1.10
Validate  val_accuracy-->37.22 val_loss-->1.09
Epoch 5 / 5
Train  accuracy->36.70 loss->1.10
Validate  val_accuracy-->37.56 val_loss-->1.09
Test Accuracy--->85.10 Test Loss--->1.00


####DATASET: MINIST06

####MODEL NAME: AlexNet

####NAME CODE: MINIST05_AlexNet

####STATUS: With Backward Learning

In [ ]:
transition_matrix_M6_Alex_Y, train_M6_Alex_Y, val_M6_Alex_Y, test_M6_Alex_Y = MINIST(batch_size=256, name='MINIST06_AlexNet')
run_train_val(train_loader=train_M6_Alex_Y, val_loader=val_M6_Alex_Y, test_loader=test_M6_Alex_Y, model_name='AlexNet', transition_matrix=transition_matrix_M6_Alex_Y, lr=0.00015, epoch_num=10)

Code Name:  MINIST06_AlexNet
Setting: MINIST06_AlexNet activated!
【New turn...】
Epoch 1 / 10
Train  accuracy->33.88 loss->2.24
Validate  val_accuracy-->34.97 val_loss-->1.12
Epoch 2 / 10
Train  accuracy->33.83 loss->1.44
Validate  val_accuracy-->35.19 val_loss-->1.10
Epoch 3 / 10
Train  accuracy->33.67 loss->1.28
Validate  val_accuracy-->35.92 val_loss-->1.10
Epoch 4 / 10
Train  accuracy->34.63 loss->1.22
Validate  val_accuracy-->36.58 val_loss-->1.10
Epoch 5 / 10
Train  accuracy->34.42 loss->1.19
Validate  val_accuracy-->36.58 val_loss-->1.10
Epoch 6 / 10
Train  accuracy->34.68 loss->1.17
Validate  val_accuracy-->36.86 val_loss-->1.10
Epoch 7 / 10
Train  accuracy->34.74 loss->1.16
Validate  val_accuracy-->36.19 val_loss-->1.10
Epoch 8 / 10
Train  accuracy->34.88 loss->1.15
Validate  val_accuracy-->36.61 val_loss-->1.10
Epoch 9 / 10
Train  accuracy->34.70 loss->1.14
Validate  val_accuracy-->37.64 val_loss-->1.10
Epoch 10 / 10
Train  accuracy->34.48 loss->1.14
Validate  val_accuracy-->3